In [5]:
import re
import numpy as np
import pandas as pd
import math
import pickle
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [6]:
stopwords=['కూడా', 'ఉన్నారు', 'ఎవరైనా', 'ఎప్పుడు', 'ఎక్కడైనా', 'అందుబాటులో', 'ఒక ప్రక్కన', 'సంబంధం', 'మాత్రమే', 'అడగండి', 'గా', 'మధ్య', 'ప్రకారం', 'అనుమతించు', 'మెచ్చుకో', 'వద్ద', 'తగిన', 'అడ్డంగా', 'అడగడం', 'ఇప్పటికే', 'కనిపిస్తాయి', 'కాదు', 'మరియు', 'మరొక', 'అనుగుణంగా', 'అయితే', 'చేయగలిగింది', 'దాదాపు', 'గురించి', 'పై', 'వెంట', 'వేరుగా', 'చుట్టూ', 'దూరంగా', 'ఏ', 'నిజంగా', 'అనుమతిస్తుంది', 'అందరూ', 'ఎవరో ఒకరు', 'వ్యతిరేకంగా', 'ఏదైనా', 'తర్వాత', 'మళ్ళీ', 'ఏమైనప్పటికి', 'ఒక']

In [7]:
vowels=set("అ ఆ ఇ ఈ ఉ ఊ ఋ ౠ ఌ ౡ ఎ ఏ ఐ ఒ ఓ ఔ అం అః".split())

conso="క	ఖ	గ	ఘ	ఙ	చ	ఛ	జ	ఝ	ఞ	ట	ఠ	డ	ఢ	ణ	త	థ	ద	ధ	న	ప	ఫ	బ	భ	మ	య	ర	ల	వ	శ	ష	స	హ	ళ	క్ష	ఱ"

matra = "ఀ	ఁ	ం	ః	ఄ	ఽ	ా	ి	ీ	ు	ూ	ృ	ౄ	ె	ే	ై	ొ	ో	ౌ	్	ౕ	ౖ	ౢ	ౣ"

halantha='్'

In [8]:
consonants=set(conso.split("\t"))
matra = set(matra.split("\t"))

vowels = list(vowels)
consonants=list(consonants)
matra=list(matra)

alphabets = vowels+consonants

In [9]:
print(alphabets)

['ఆ', 'ఏ', 'ౠ', 'ఒ', 'ఌ', 'అ', 'ఓ', 'ఈ', 'ఋ', 'అం', 'ఇ', 'ఐ', 'ఔ', 'ఎ', 'అః', 'ఊ', 'ఉ', 'ౡ', 'ల', 'ఢ', 'ణ', 'క్ష', 'ఙ', 'ఛ', 'ట', 'ఱ', 'ఞ', 'య', 'ద', 'మ', 'డ', 'శ', 'ప', 'స', 'ఘ', 'ఝ', 'హ', 'ర', 'గ', 'ధ', 'ష', 'ళ', 'వ', 'భ', 'జ', 'థ', 'ఖ', 'బ', 'చ', 'త', 'న', 'ఠ', 'ఫ', 'క']


In [10]:
#data=pd.read_csv('../hindi.csv')
data=pd.read_csv('../telugu.csv')

In [11]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())


In [12]:
with open('pickles/df.pkl','rb') as file:
    DF=pickle.load(file)
    file.close()
    
with open('pickles/file_idx.pkl','rb') as file:
    file_idx=pickle.load(file)
    file.close()
    
with open('pickles/doc_words.pkl','rb') as file:
    doc_words=pickle.load(file)
    file.close()
    
with open('pickles/doc_norm.pkl','rb') as file:
    doc_norm=pickle.load(file)
    file.close()

In [13]:
def tfidf(query,counter=10):
    text = remove_special_characters(query)
    words = text.split()
    words=[word for word in words if word not in stopwords]
    words=[word for word in words if word in DF.keys()]

    q=[]
    q_norm=0
    for w in words:
        tf_idf=(words.count(w)*math.log(len(file_idx)/DF[w]))
        q.append(tf_idf)
        q_norm+=tf_idf**2
    q_norm=math.sqrt(q_norm)

    q=np.array(q)/q_norm

    score={}

    for i in range(len(file_idx)):
        doc_v=[]
        for w in words:
            tf_idf=(doc_words[i].count(w)*math.log(len(file_idx)/DF[w]))
            doc_v.append(tf_idf)
        doc_v=np.array(doc_v)/doc_norm[i]
        score[i]=np.dot(q,doc_v)

    score=sorted(score.items(),key=lambda x:x[1],reverse=True)
    verse_index=[]
    for i in score:
        if counter== 0:
            break
        verse_index.append(i[0])
        counter-=1
    return verse_index
    

In [14]:
verse_index=tfidf('నాకు చాలా సమస్యలు ఉన్నాయి')
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
47,2,8,न हि प्रपश्यामि ममापनुद्या\n\nद्यच्छोकमुच्छोषण...,"ఎప్పుడైనా మనము దుఃఖం లో మునిగిపోయినప్పుడు, మన ..."
589,17,2,श्री भगवानुवाचत्रिविधा भवति श्रद्धा देहिनां सा...,"ఎవ్వరూ కూడా శ్రద్ధ/విశ్వాసము లేకుండా ఉండరు, ఎం..."
527,14,11,सर्वद्वारेषु देहेऽस्मिन्प्रकाश उपजायते।ज्ञानं ...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
528,14,12,लोभः प्रवृत्तिरारम्भः कर्मणामशमः स्पृहा।रजस्ये...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
529,14,13,अप्रकाशोऽप्रवृत्तिश्च प्रमादो मोह एव च।तमस्येत...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
596,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,ఎప్పుడైతే శాకాహార పదార్ధములను మితిమీరిన కారం/మ...
410,11,4,मन्यसे यदि तच्छक्यं मया द्रष्टुमिति प्रभो।\n\n...,"ఇంతకు క్రితం శ్లోకంలో, అర్జునుడు సర్వేశ్వరుని ..."
316,8,14,अनन्यचेताः सततं यो मां स्मरति नित्यशः।\n\nतस्य...,భగవద్గీత అంతటా శ్రీ కృష్ణుడు చాలా సార్లు భక్తి...
595,17,8,आयुःसत्त्वबलारोग्यसुखप्रीतिविवर्धनाः।रस्याः स्...,"14వ అధ్యాయము, 6వ శ్లోకములో, శ్రీ కృష్ణుడు సత్త..."
515,13,34,यथा प्रकाशयत्येकः कृत्स्नं लोकमिमं रविः।क्षेत्...,"ఆత్మ, తను ఉన్న దేహమును చైతన్యముచే తేజోమయం చేసి..."


In [17]:
data.loc[47]["Commentary"]

'ఎప్పుడైనా మనము దుఃఖం లో మునిగిపోయినప్పుడు, మన బుద్ది ఆ దుఃఖానికి మూల కారణాన్ని విశ్లేషిస్తూ ఉంటుంది. ఎపుడైతే ఇక ఎక్కువ ఆలోచించలేదో, అప్పుడు మానసికంగా కుంగిపోవటం మొదలవుతుంది. అర్జునుడి సమస్యలు అతని అల్పమైన బుద్ది కన్నా పెద్దవిగా పరిణమించటంతో, అతన్ని శోక సముద్రం నుండి కాపాడటానికి తనకున్న భౌతిక జ్ఞానం సరిపోదు. శ్రీ కృష్ణుడిని గురువుగా స్వీకరించిన తరువాత తన దయనీయ స్థితిని వెల్లడిచేయుచూ మనస్సు లో ఉన్నదంతా చెప్తున్నాడు, అర్జునుడు.   అర్జునుడి పరిస్థితి అతనొక్కడిదే కాదు. జీవిత ప్రయాణం లో సాగిపోతున్నప్పుడు అప్పుడప్పుడు మనకు ఎదురయ్యేదే. మనకు సంతోషం కావాలి, కానీ దుఃఖం కలుగుతుంటుంది; మనకు జ్ఞానం కావాలి; కాని అజ్ఞానపు మేఘాల్ని తొలగించుకోలేము; పరిశుద్ధమైన ప్రేమని కోరుకుంటాము కాని పదేపదే ఆశాభంగము కలుగుతుంది; మన కాలేజి పట్టాలు, నేర్చుక్కున్న విద్య మరియు లౌకిక పాండిత్యములు జీవితం లో ఎదురయ్యే జటిల సమస్యలకు పరిష్కారం చూపలేవు. మనకు జీవితపు క్లిష్టమైన సమస్యలను పరిష్కరించటానికి ఆధ్యాత్మిక జ్ఞానం అవసరం. మహోన్నత స్థితి లో ఉన్న నిజమైన గురువు లభించినప్పుడు, మనకు వారి నుండి నేర్చుకునే అణకువ, వినయం ఉంటే ఆ యొక్క